#CS6910: Deep Learning - Assignment 1: Fashion MNIST dataset

Shibobrota Das (CS20M059) | Abhishek Kumar (CS20M007)

In [24]:
# !pip install wandb -qqq
import wandb
wandb.init(project="assignment-1")

Accuracy,0.75531
Error,0.24469
_runtime,168
_timestamp,1616704718
_step,1079


Accuracy,▁▂▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
Error,█▇▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from scipy.special import expit, softmax
from sklearn.model_selection import train_test_split

In [26]:
(train_x, train_y), (test_x, test_y)= fashion_mnist.load_data()

In [27]:
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [28]:
#Reshape 2D Matrix into 1D Vector
train_x = np.array(train_x.reshape(len(train_x), 28 ** 2, 1))
test_x = np.array(test_x.reshape(len(test_x), 28 ** 2, 1))

#Normalizing data
train_x = train_x/255.0
test_x = test_x/255.0

#Splitting Data
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size = 0.9, test_size = 0.1)

In [29]:
np.shape(train_x[0])

(784, 1)

In [30]:
#Activation function Def

def relu(z):
  return x * (x > 0)                                           #learnt from https://stackoverflow.com/questions/32109319/how-to-implement-the-relu-function-in-numpy

def sigmoid(z):
  return expit(z)

def tanh(z):
  return np.tanh(z)

In [31]:
#Activation function derivative Def

def grad_tanh(x):
  return 1-(tanh(x) ** 2)

def grad_sigmoid(x):
  sig_x = sigmoid(x)
  return sig_x*(1-sig_x)

def grad_relu(x):
  temp = 1. * (x > 0)                                          #learnt from https://stackoverflow.com/questions/32109319/how-to-implement-the-relu-function-in-numpy
  return temp

In [32]:
#Type Initialization
RANDOM = "random"
XAVIER = "xavier"
ZEROES = "zeroes"

#Definitions

def init_weights(type: str, layers):
    W = {}
    for i in range(1, len(layers)):
      if type == RANDOM:
        W[i] = np.random.normal(0, 0.5, size=(layers[i], layers[i-1]))
      elif type == XAVIER:
        W[i] = np.random.uniform(-(1/np.sqrt(layers[i])), (1/np.sqrt(layers[i])), size=(layers[i], layers[i-1]))
      else:
        W[i] = np.zeros((layers[i], layers[i-1]))
    return W

def init_bias(type: str, layers):
    B = {}
    for i in range(1, len(layers)):
      if type == RANDOM:
        B[i] = np.random.randn(layers[i], 1)
      else:
        B[i] = np.zeros((layers[i], 1))
    return B

In [33]:
def init_params(type: str, layers):
  return init_weights(type, layers), init_bias(type, layers)

In [34]:
# #Initialize the layer info - number of neurons per layer
layers = [6] + [4] * 2 + [2]

#Initialize network the parameters
W, B = init_params(RANDOM, layers)

In [35]:
#Type Initialization
SIGMOID = "sigmoid"
TAN_H = "tanh"
RELU = "relu"

#Definitions

def grad(activation: str, pre_act_vector):
  #increase precision
  pre_act_vector = np.array(pre_act_vector,dtype=np.longdouble)
  if activation == TAN_H:
    return grad_tanh(pre_act_vector)
  elif activation == SIGMOID:
    return grad_sigmoid(pre_act_vector)
  elif activation == RELU:
    return grad_relu(pre_act_vector)
  return None

def activate(activation: str, pre_act_vector):
  #increase precision
  pre_act_vector = np.array(pre_act_vector,dtype=np.longdouble)
  if activation == TAN_H:
    return tanh(pre_act_vector)
  elif activation == SIGMOID:
    return sigmoid(pre_act_vector)
  elif activation == RELU:
    return relu(pre_act_vector)
  return None

In [36]:
#Forward Propagation

def forward_propagation(data, activation, W, B, sizes):                         
  H = {0:data}
  A = {}
  L = len(layers)-1
  for k in range(1, L):
    A[k] = np.matmul(W[k],H[k-1])+B[k]
    H[k] = activate(SIGMOID, A[k])
    
  A[L] = np.matmul(W[L],H[L-1])+B[L]
  y_hat = softmax(A[L])
  
  return H, A, y_hat

In [37]:
layers = [784,64,64,64, 10]
W, B = init_params(RANDOM, layers)

In [38]:
H, A, y_hat = forward_propagation(train_x[0], SIGMOID, W, B, layers)

In [39]:
def back_propagation(y, y_hat, A, H, W, layers, activation):
  Y = np.array([1 if i == y else 0 for i in range(len(y_hat))]).reshape(np.shape(y_hat))
  dW, dB = init_params(ZEROES, layers)
  dA = init_bias(ZEROES, layers)
  dH = init_bias(ZEROES, layers)
  L = len(layers) - 1
  dA[L] = -(Y - y_hat)
  for i in range(L, 0, -1):
    dW[i] = np.dot(dA[i], H[i - 1].T)
    dB[i] = dA[i]
    if i > 1:
      dH[i - 1] = np.dot(W[i].T, dA[i])
      dA[i - 1] = np.multiply(dH[i - 1], grad(activation, A[i - 1]))
  return dW, dB

In [40]:
dW, dB = back_propagation(train_y[0],y_hat,A,H,W,layers,SIGMOID)

In [41]:
def cross_entropy_loss(label, y_hat):
  return - np.log(y_hat[label][0])

In [42]:
def squared_error_loss(label, y_hat):
  Y = np.array([1 if i == label else 0 for i in range(len(y_hat))]).reshape(np.shape(y_hat))
  loss = (Y - y_hat) ** 2
  return np.average(loss)

In [43]:
squared_error_loss(train_y[0], y_hat)

0.12763035647215507

In [49]:
def momentum_GD(data,label,activation,epochs,layers,eta,init_type,batch_size=1):

  gamma = 0.9
  W, B = init_params(init_type, layers)
  v_W, v_B = init_params(ZEROES, layers)
  dW, dB = init_params(ZEROES, layers)
  priv_W, priv_B = init_params(ZEROES, layers)
  L = len(layers) - 1
  counter, ce, se = 0, 0, 0
  loss_list = {"CE": [], "SE": []}
  accuracy = 0
  for n in range(epochs):
    for k in range(len(data)):
      H, A, y_hat = forward_propagation(data[k], activation, W, B, layers)
      w, b = back_propagation(label[k],y_hat,A,H,W,layers,activation)
      for i in range(1, L+1):
        dW[i] += w[i]
        dB[i] += b[i]

      tem = np.argmax(y_hat, axis=0)[0]
      if tem == label[k]:
        accuracy += 1
      counter += 1
      if counter % batch_size == 0:
        for i in range(1, L+1):
          v_W[i] = gamma * priv_W[i] + eta * dW[i]
          v_B[i] = gamma * priv_B[i] + eta * dB[i]
          W[i] -= v_W[i]
          B[i] -= v_B[i]
        priv_W = v_W
        priv_B = v_B
        dW, dB = init_params(ZEROES, layers)
        ce = cross_entropy_loss(label[k], y_hat)
        se = squared_error_loss(label[k], y_hat)
        print("Accuracy", accuracy/counter)
        wandb.log({"MGD Accuracy": accuracy/counter, "MGD Error": 1-(accuracy/counter), "MGD Cross Entropy": float(ce)})
    
    print("Epoch", n,"Accuracy", accuracy/counter)
  return W, B, loss_list

In [48]:
layers = [784,64,64,64, 10]
eta = 0.001
W, B, loss = momentum_GD(train_x, train_y, SIGMOID, 10, layers, eta, RANDOM, 50)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()